# Setup Visdom

Install it with:

`pip install visdom`

Start the server:

`python -m visdom.server`

Visdom now can be accessed at http://localhost:8097 in the browser.


# LDA Training Visualization

To monitor the LDA training, a list of Metrics can be passed to the LDA function call for plotting their values live as the training progresses.  

Let's plot the training stats for an LDA model being trained on Lee corpus. We will use the four evaluation metrics available for topic models in gensim: Coherence, Perplexity, Topic diff and Convergence. (using separate hold_out and test corpus for evaluating the perplexity)

In [2]:
import os
import re
import gensim
from gensim.models import ldamodel
from gensim.corpora.dictionary import Dictionary


# Set file names for train and test data
test_data_dir = os.path.join(gensim.__path__[0], 'test', 'test_data')
lee_train_file = os.path.join(test_data_dir, 'lee_background.cor')
lee_test_file = os.path.join(test_data_dir, 'lee.cor')

def read_corpus(fname):
    texts = []
    with open(fname, encoding="ISO-8859-1") as f:
        for line in f:
            # lower case all words
            lowered = line.lower()
            # remove punctuation and split into seperate words
            words = re.compile('\w+').findall(lowered)
            texts.append(words)
    return texts

training_texts = read_corpus(lee_train_file)
eval_texts = read_corpus(lee_test_file)

# Split test data into hold_out and test corpus
holdout_texts = eval_texts[:25]
test_texts = eval_texts[25:]

training_dictionary = Dictionary(training_texts)
holdout_dictionary = Dictionary(holdout_texts)
test_dictionary = Dictionary(test_texts)

training_corpus = [training_dictionary.doc2bow(text) for text in training_texts]
holdout_corpus = [holdout_dictionary.doc2bow(text) for text in holdout_texts]
test_corpus = [test_dictionary.doc2bow(text) for text in test_texts]

In [18]:
from gensim.models.callbacks import CoherenceMetric, DiffMetric, PerplexityMetric, ConvergenceMetric

# define perplexity callback for hold_out and test corpus
pl_holdout = PerplexityMetric(corpus=holdout_corpus, logger="visdom", title="Perplexity (hold_out)")
pl_test = PerplexityMetric(corpus=test_corpus, logger="visdom", title="Perplexity (test)")

# define other remaining metrics available
ch_umass = CoherenceMetric(corpus=training_corpus, coherence="u_mass", logger="visdom", title="Coherence (u_mass)")
ch_cv = CoherenceMetric(corpus=training_corpus, texts=training_texts, coherence="c_v", logger="visdom", title="Coherence (c_v)")
diff_kl = DiffMetric(distance="kullback_leibler", logger="visdom", title="Diff (kullback_leibler)")
convergence_kl = ConvergenceMetric(distance="kullback_leibler", logger="visdom", title="Convergence (kullback_leibler)")

callbacks = [pl_holdout, pl_test, ch_umass, ch_cv, diff_kl, convergence_kl]

# training LDA model
model = ldamodel.LdaModel(corpus=training_corpus, id2word=training_dictionary, passes=20, num_topics=5, eval_every=None, callbacks=callbacks)

When the model is set for training, you can open http://localhost:8097 to see the training progress.

In [3]:
# to get a metric value on a trained model
print(CoherenceMetric(corpus=training_corpus, coherence="u_mass").get_value(model=model))

-0.259766196856


The four types of graphs which are plotted for LDA:

**Coherence**

Coherence measures are generally based on the idea of computing the sum of pairwise scores of top *n* top words w<sub>1</sub>, ...,w<sub>n</sub> used to describe the topic. There are four coherence measure available in gensim: `u_mass, c_v, c_uci, c_npmi`. A good model will generate coherent topics, i.e., topics with high topic coherence scores. Good topics can be described by a short label based on the topic terms they spit out. 

<img src="Coherence.gif">

Now, this graph along with the others explained below, can be used to decide if it's time to stop the training. We     can see if the value stops changing after some epochs and that we are able to get the highest possible coherence       of our model.  


**Perplexity**

Perplexity is a measurement of how well a probability distribution or probability model predicts a sample. In LDA, topics are described by a probability distribution over vocabulary words. So, perplexity can be used to evaluate the topic-term distribution output by LDA.

<img src="Perplexity.gif">

For a good model, perplexity should be low.


**Topic Difference**

Topic Diff calculates the distance between two LDA models. This distance is calculated based on the topics, by either using their probability distribution over vocabulary words (kullback_leibler, hellinger) or by simply using the common vocabulary words between the topics from both model.

<img src="Diff.gif">

In the heatmap, X-axis define the Epoch no. and Y-axis define the distance between identical topics from consecutive epochs. For ex. a particular cell in the heatmap with values (x=3, y=5, z=0.4) represent the distance(=0.4) between the topic 5 from 3rd epoch and topic 5 from 2nd epoch. With increasing epochs, the distance between the identical topics should decrease.
  
  
**Convergence**

Convergence is the sum of the difference between all the identical topics from two consecutive epochs. It is basically the sum of column values in the heatmap above.

<img src="Convergence.gif">

The model is said to be converged when the convergence value stops descending with increasing epochs.

# Training Logs

We can also log the metric values after every epoch to the shell apart from visualizing them in Visdom. The only change is to define `logger="shell"` instead of `"visdom"` in the input callbacks.

In [9]:
import logging
from gensim.models.callbacks import CoherenceMetric, DiffMetric, PerplexityMetric, ConvergenceMetric

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

# define perplexity callback for hold_out and test corpus
pl_holdout = PerplexityMetric(corpus=holdout_corpus, logger="shell", title="Perplexity (hold_out)")
pl_test = PerplexityMetric(corpus=test_corpus, logger="shell", title="Perplexity (test)")

# define other remaining metrics available
ch_umass = CoherenceMetric(corpus=training_corpus, coherence="u_mass", logger="shell")
diff_kl = DiffMetric(distance="kullback_leibler", logger="shell")
convergence_kl = ConvergenceMetric(distance="kullback_leibler", logger="shell")

callbacks = [pl_holdout, pl_test, ch_umass, diff_kl, convergence_kl]

# training LDA model
model = ldamodel.LdaModel(corpus=training_corpus, id2word=training_dictionary, passes=3, num_topics=5, eval_every=None, callbacks=callbacks)

INFO:gensim.models.ldamodel:using symmetric alpha at 0.2
INFO:gensim.models.ldamodel:using symmetric eta at 0.00013900472616068947
INFO:gensim.models.ldamodel:using serial LDA version on this node
INFO:gensim.models.ldamodel:running online (multi-pass) LDA training, 5 topics, 3 passes over the supplied corpus of 300 documents, updating model once every 300 documents, evaluating perplexity every 0 documents, iterating 50x with a convergence threshold of 0.001000
INFO:gensim.models.ldamodel:PROGRESS: pass 0, at document #300/300
INFO:gensim.models.ldamodel:topic #0 (0.200): 0.051*"the" + 0.020*"in" + 0.019*"to" + 0.018*"of" + 0.015*"a" + 0.013*"and" + 0.011*"is" + 0.011*"for" + 0.009*"he" + 0.008*"says"
INFO:gensim.models.ldamodel:topic #1 (0.200): 0.048*"the" + 0.030*"to" + 0.022*"in" + 0.021*"and" + 0.020*"a" + 0.019*"of" + 0.010*"s" + 0.009*"for" + 0.008*"that" + 0.008*"have"
INFO:gensim.models.ldamodel:topic #2 (0.200): 0.023*"the" + 0.022*"to" + 0.018*"of" + 0.015*"in" + 0.013*"and"

The metric values can also be accessed from the model instance for custom uses.

In [10]:
model.metrics

defaultdict(list,
            {'Coherence': [-0.25410927592387839,
              -0.24879204118159887,
              -0.25756427989868341],
             'Convergence': [4.1214964252266926,
              3.5441769077766914,
              3.031822101038804],
             'Diff': [array([ 0.79628357,  0.90363194,  0.55469714,  1.        ,  0.86688377]),
              array([ 0.83029118,  0.72960219,  1.        ,  0.22719304,  0.75709049]),
              array([ 0.82668066,  0.50774819,  1.        ,  0.19109239,  0.50630086])],
             'Perplexity (hold_out)': [400318.44116470998,
              231516.22056950352,
              185972.72652968348],
             'Perplexity (test)': [1242745.7210251174,
              666335.54087631544,
              516819.88515415508]})